In [1]:
import pandas as pd
import numpy as np
import os
import networkx as nx
import sys
sys.path.insert(0, '..')

In [2]:
import warnings
warnings.filterwarnings('error')

# Load Data

In [3]:
eighteenS = pd.read_csv('../data/cleaned/NCOG_18sV4_filtered_asv_count_tax.tsv', sep='\t', index_col=0)
sixteenS = pd.read_csv('../data/cleaned/NCOG_21_16sV4_redo2_filtered_asv_count_tax.tsv', sep='\t', index_col=0)
meta = pd.read_csv('../data/input/NCOG_sample_log_DNA_stvx_meta_2014-2020_mod.tsv', sep='\t', index_col=0)
meta.index = 'X' + meta.index

In [4]:
meta.columns

Index(['sample_num', 'Cruise', 'Event', 'Order_Occ', 'Sta_ID', 'Cast_Type',
       'Cardinal_Sta', 'Station_Notes', 'Bottle', 'Assoc_Bottle', 'Depthm',
       'Bottle_Notes', 'NCOG_DNA', 'NCOG_RNA', 'DNA_VolFilt', 'RNA_VolFilt',
       'Pump_Speed', 'DNA_RNA_Vol_Notes', 'Filt_Str', 'Filt_End',
       'Filt_Str_Notes', 'Filt_End_Notes', 'Gly.Sample', 'Sterivex_Notes',
       'std_18S', 'Date', 'Time', 'DateTime', 'Lat_Dec', 'Lon_Dec', 'Distance',
       'NoCCSamples', 'Del_Depth', 'CC_Depth', 'T_degC', 'Salnty', 'STheta',
       'O2ml_L', 'PO4ug', 'SiO3ug', 'NO3ug', 'NH3ug', 'ChlorA', 'Phaeop',
       'RecInd', 'Code_CCE', 'IntChl', 'IntC14', 'NCDepth', 'MLD_Sigma',
       'season', 'year', 'sample_type', 'sample_type2', 'siex', 'exclude'],
      dtype='object')

## Get subgraph of the supergraph based on which nodes are present
I believe this is the implementation done in the Chafron paper to get the degree/strength/weight/density/transitivity X environment variable values correlation matrix

In [5]:
gph = nx.read_gml('../data/graph/NCOG_network_output_v4.gml')

In [6]:
def get_average_strength(local_nw_gph):
    edgeweights = nx.get_edge_attributes(local_nw_gph, 'weight')
    edges = pd.DataFrame({
        'Node1': pd.Series(edgeweights.keys(), dtype=object).apply(lambda x: x[0]),
        'Node2': pd.Series(edgeweights.keys(), dtype=object).apply(lambda x: x[1]),
        'Weight': list(edgeweights.values())
    })
    edges_combined = pd.concat([edges, edges.rename({'Node1': 'Node2', 'Node2': 'Node1'}, axis=1)])
    return edges_combined.groupby('Node1')['Weight'].sum().mean()

In [7]:
all_data = pd.concat([sixteenS, eighteenS], axis=1)
gph_stats = []
for sample, row in all_data.iterrows():
    included_nodes = list(row[row != 0].index)
    sample_gph = gph.subgraph(included_nodes)
    try:
        sample_stats = {
            'Mean degree': pd.Series(np.array(nx.degree(sample_gph))[:,1]).astype(int).mean(),
            'Mean strength': get_average_strength(sample_gph),
            'Mean weight': pd.Series(nx.get_edge_attributes(sample_gph, 'weight').values(), dtype=np.float64).mean(),
            'Edge density': nx.density(sample_gph), # nx.density(local_nw) # Is this edge density? Or a different density?
            'Transitivity': nx.transitivity(sample_gph)
        }
    except:
        print(nx.get_edge_attributes(sample_gph, 'weight'))
        print(row)
        print(included_nodes)
        break
    gph_stats.append(sample_stats)
gph_stats_df = pd.DataFrame(gph_stats, index=all_data.index)

In [8]:
# create new column with station ID and sample type
meta['Sta_ID_with_sample_type'] = meta['Sta_ID'] + '; ' + meta['sample_type'].astype(str)

In [9]:
all_data = pd.concat([sixteenS, eighteenS], axis=1)
gph_stats = []
included_stations = []
for station_id in sorted(meta['Sta_ID_with_sample_type'].unique()):
    included_samples = meta[meta['Sta_ID_with_sample_type'] == station_id].index.values
    included_rows = all_data.loc[all_data.index.isin(included_samples)]
    if len(included_rows) == 0:
        print('No matching samples found for station', station_id)
        continue
    included_stations.append(station_id)
    included_nodes = pd.Series(0, index=all_data.columns)
    for sample_id, row in included_rows.iterrows():
        # include every OTU that has an abundance of >0 in any of the samples
        # (prokaryotes and eukaryotes calculated separately)
        prokaryotes = row[row.index.str.contains('potu')]
        eukaryotes = row[row.index.str.contains('eotu')]
        included_potus = prokaryotes[prokaryotes > 0].index
        included_eotus = eukaryotes[eukaryotes > 0].index
        included_nodes.loc[included_potus] = 1
        included_nodes.loc[included_eotus] = 1
    included_nodes = list(included_nodes[included_nodes > 0].index)
    sample_gph = gph.subgraph(included_nodes)
    print('Station', station_id, '\t', len(sample_gph.nodes()), 'Nodes;', len(sample_gph.edges()), 'Edges')
    try:
        sample_stats = {
            'Mean degree': pd.Series(np.array(nx.degree(sample_gph))[:,1]).astype(int).mean(),
            'Mean strength': get_average_strength(sample_gph),
            'Mean weight': pd.Series(nx.get_edge_attributes(sample_gph, 'weight').values(), dtype=np.float64).mean(),
            'Edge density': nx.density(sample_gph), # nx.density(local_nw) # Is this edge density? Or a different density?
            'Transitivity': nx.transitivity(sample_gph)
        }
    except:
        print(nx.get_edge_attributes(sample_gph, 'weight'))
        print(row)
        print(included_nodes)
        break
    gph_stats.append(sample_stats)
gph_stats_df = pd.DataFrame(gph_stats, index=included_stations)

Station 060.0 060.0; DCM 	 761 Nodes; 3512 Edges
Station 060.0 060.0; Surf 	 686 Nodes; 3181 Edges
No matching samples found for station 060.0 070.0; DCM
No matching samples found for station 060.0 070.0; Surf
Station 060.0 080.0; DCM 	 1070 Nodes; 5810 Edges
Station 060.0 080.0; Surf 	 833 Nodes; 4169 Edges
No matching samples found for station 060.0 090.0; Surf
No matching samples found for station 063.3 052.0; DCM
No matching samples found for station 063.3 052.0; Surf
No matching samples found for station 063.3 055.0; DCM
No matching samples found for station 063.3 055.0; Surf
No matching samples found for station 063.3 060.0; DCM
No matching samples found for station 063.3 060.0; Surf
Station 063.3 070.0; DCM 	 635 Nodes; 2811 Edges
Station 063.3 070.0; Surf 	 517 Nodes; 2199 Edges
No matching samples found for station 063.3 080.0; DCM
No matching samples found for station 063.3 080.0; Surf
Station 063.3 090.0; DCM 	 889 Nodes; 4222 Edges
Station 063.3 090.0; Surf 	 405 Nodes; 165

# Group by station ID get correlation with environmental variables
- could also group by sample_type additionally

In [10]:
meta_grouped_by_Sta_ID = (
    meta[['Sta_ID_with_sample_type', 'Lon_Dec', 'Lat_Dec', 'T_degC', 'Salnty', 'O2ml_L',
          'ChlorA', 'PO4ug', 'NO3ug', 'SiO3ug', 'NH3ug', 'NCDepth',
          'CC_Depth', 'Del_Depth', 'MLD_Sigma', 'Phaeop', 'STheta']]
    .groupby(
        'Sta_ID_with_sample_type'
    ).median()
)

In [11]:
meta_grouped_by_Sta_ID

,Lon_Dec,Lat_Dec,T_degC,Salnty,O2ml_L,ChlorA,PO4ug,NO3ug,SiO3ug,NH3ug,NCDepth,CC_Depth,Del_Depth,MLD_Sigma,Phaeop,STheta
Sta_ID_with_sample_type,,,,,,,,,,,,,,,,
060.0 060.0; DCM,-123.601500,37.609080,12.561000,32.847801,6.1890,0.963,0.340,0.31,2.20,0.260,43.751196,41.0,-1.0,11.422473,0.303,24.814119
060.0 060.0; Surf,-123.601500,37.609080,13.397000,32.838299,6.3300,0.684,0.280,0.00,2.02,0.000,43.751196,10.0,0.0,11.422473,0.062,24.640940
060.0 070.0; DCM,-124.328900,37.279730,12.916000,32.824501,6.1140,0.497,NaN,NaN,NaN,NaN,NaN,20.0,0.0,14.853170,0.156,24.726110
060.0 070.0; Surf,-124.328900,37.279730,13.100000,32.818699,6.1230,0.318,NaN,NaN,NaN,NaN,NaN,10.0,0.0,14.853170,0.203,24.684900
060.0 080.0; DCM,-125.046170,36.945820,13.781000,33.146099,5.9490,0.687,0.260,0.00,1.95,0.000,88.137255,85.0,0.0,16.489248,0.162,24.805401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
093.3 110.0; DCM,-122.919030,30.177480,14.474000,33.405800,5.4660,0.240,0.370,0.90,3.22,0.020,103.196721,107.0,0.0,34.539839,0.144,24.864349
093.3 110.0; Surf,-122.919030,30.177480,17.316999,33.363098,5.4810,0.080,0.260,0.00,1.85,0.060,103.196721,16.0,0.0,34.539839,0.013,24.185049
093.3 120.0; DCM,-123.586470,29.847470,14.290000,33.415901,5.5540,0.216,0.340,0.44,2.71,0.050,115.720000,113.0,0.0,24.612560,0.161,24.947470


In [12]:
gph_stats_with_meta = pd.concat([gph_stats_df, meta_grouped_by_Sta_ID], axis=1).dropna()
gph_stats_with_meta

,Mean degree,Mean strength,Mean weight,Edge density,Transitivity,Lon_Dec,Lat_Dec,T_degC,Salnty,O2ml_L,...,PO4ug,NO3ug,SiO3ug,NH3ug,NCDepth,CC_Depth,Del_Depth,MLD_Sigma,Phaeop,STheta
060.0 060.0; DCM,9.229961,1.057696,0.113991,0.012145,0.081861,-123.601500,37.60908,12.561000,32.847801,6.1890,...,0.340,0.31,2.20,0.260,43.751196,41.0,-1.0,11.422473,0.3030,24.814119
060.0 060.0; Surf,9.274052,1.105229,0.118827,0.013539,0.088155,-123.601500,37.60908,13.397000,32.838299,6.3300,...,0.280,0.00,2.02,0.000,43.751196,10.0,0.0,11.422473,0.0620,24.640940
060.0 080.0; DCM,10.859813,1.166727,0.107134,0.010159,0.092907,-125.046170,36.94582,13.781000,33.146099,5.9490,...,0.260,0.00,1.95,0.000,88.137255,85.0,0.0,16.489248,0.1620,24.805401
060.0 080.0; Surf,10.009604,1.143867,0.113454,0.012031,0.116434,-125.049895,36.94575,13.588000,33.003351,6.1080,...,0.230,0.00,1.97,0.000,88.137255,10.0,0.0,15.725447,0.0890,24.725175
063.3 070.0; DCM,8.853543,1.074483,0.120789,0.013965,0.084163,-123.910465,36.70869,12.854000,33.081450,6.0995,...,0.250,0.00,1.50,0.000,39.514187,27.5,0.0,15.454565,0.3185,24.937600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
093.3 100.0; Surf,14.414723,1.516109,0.105069,0.007477,0.076323,-122.261170,30.51307,18.443001,33.450901,5.4630,...,0.280,0.00,1.85,0.000,99.368421,12.0,0.0,41.583355,0.0300,24.048380
093.3 110.0; DCM,12.953811,1.398315,0.107697,0.007483,0.086443,-122.919030,30.17748,14.474000,33.405800,5.4660,...,0.370,0.90,3.22,0.020,103.196721,107.0,0.0,34.539839,0.1440,24.864349
093.3 110.0; Surf,11.807469,1.327631,0.112347,0.009807,0.103337,-122.919030,30.17748,17.316999,33.363098,5.4810,...,0.260,0.00,1.85,0.060,103.196721,16.0,0.0,34.539839,0.0130,24.185049
093.3 120.0; DCM,13.903602,1.562352,0.112370,0.007368,0.073352,-123.586470,29.84747,14.290000,33.415901,5.5540,...,0.340,0.44,2.71,0.050,115.720000,113.0,0.0,24.612560,0.1610,24.947470


In [13]:
gph_stats_with_meta.to_csv('../data/analysis/NCOG_local_network_stats_with_envvar.csv')

### Make correlation plot (cwd changes working directory to the root of the repo)

In [14]:
import subprocessx
subprocess.run(['Rscript', 'src/flashweave_local_network_correlation.R'], cwd='..')

ModuleNotFoundError: No module named 'subprocessx'